# Conformance Checking with a LTL model

The class `ProcessMiningTasks.ConformanceChecking.LTLAnalyzer.LTLAnalyzer` provides a way to check if the log conforms to a Linear Temporal Logic (LTL) formula. The formula can be a provided by the user as a string, note that we adopted the LTL syntax [here](http://ltlf2dfa.diag.uniroma1.it/ltlf_syntax). In addition, we also provide the following set of LTL templates:

1. `eventually_a`;
2. `next_a`;
3. `eventually_a_then_b`;
4. `eventually_a_or_b`;
5. `eventually_a_next_b`;
6. `eventually_a_then_b_then_c`;
7. `eventually_a_next_b_next_c`;

and the following [Target-Branched DECLARE templates](https://www.sciencedirect.com/science/article/pii/S0306437915001271):

1. `responded_existence`;
2. `response`;
3. `alternate_response`;
4. `chain_response`;
5. `precedence`;
6. `alternate_precedence`;
7. `chain_precedence`;
8. `not_responded_existence`;
9. `not_response`;
10. `not_precedence`;
11. `not_chain_response`;
12. `not_chain_precedence`.

First of all we now import a `xes` log.

In [1]:
import sys
import os
import pathlib

SCRIPT_DIR = pathlib.Path("../../../", "src").resolve()
sys.path.append(os.path.dirname(SCRIPT_DIR))

from src.Declare4Py.D4PyEventLog import D4PyEventLog

log_path = os.path.join("../../../", "tests", "test_logs","Sepsis Cases.xes.gz")
event_log = D4PyEventLog()
event_log.parse_xes_log(log_path)

parsing log, completed traces ::   0%|          | 0/1050 [00:00<?, ?it/s]

The next step is to create an LTLModel from a string with the class `ProcessModels.LTLModel.LTLModel` and pass it to the LTL conformance checker implemented in the `src.Declare4Py.ProcessMiningTasks.ConformanceChecking.LTLAnalyzer.LTLAnalyzer` class. The corresponding method `run` will do the job and return a Pandas dataframe. This dataframe contains the traces ids in the first column and the results of the conformance checking in the second column. Here we show an example with the `F(ER Triage)` formula.

In [4]:
from src.Declare4Py.ProcessModels.LTLModel import LTLModel
from src.Declare4Py.ProcessMiningTasks.ConformanceChecking.LTLAnalyzer import LTLAnalyzer

model = LTLModel()
model.parse_from_string("F(CRP)")

analyzer = LTLAnalyzer(event_log, model)
df = analyzer.run()
df

,case:concept:name,accepted
0,A,True
1,B,True
2,C,True
3,D,True
4,E,True
...,...,...
1045,HNA,True
1046,INA,False
1047,JNA,False
1048,KNA,True


## Conformance Checking with LTL Templates

Declare4Py offers some LTL templates in the `src.Declare4Py.ProcessModels.LTLModel.LTLTemplate` class. You just need to instantiate a single template by passing the template name to the `LTLTemplate` class and then filling the template with some proper activites to obtain an `LTLModel` and run the LTL checker.

The `LTLTemplate`'s function `fill_template`, which expects a list with the parameters equal to those expected by the template and returns an `LTLModel` object containing the parsed formula of the template. You can obtain the LTL formula of the template by accessing the `formula` attribute of the `LTLModel` object.

### `eventually_a`

This is a unary template taking one activity as input. The corresponding LTL formula is `F(a)`.

In [5]:
from src.Declare4Py.ProcessModels.LTLModel import LTLTemplate

template: LTLTemplate = LTLTemplate('eventually_activity_a')
model: LTLModel = template.fill_template(['CRP'])
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
data

Formula: F(crp)


,case:concept:name,accepted
0,A,True
1,B,True
2,C,True
3,D,True
4,E,True
...,...,...
1045,HNA,True
1046,INA,False
1047,JNA,False
1048,KNA,True


### `next_a`
This is a unary template taking one activity as input. The corresponding LTL formula is `X(a)`.

In [6]:
template = LTLTemplate('next_a')
model = template.fill_template(['ER Triage'])
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
data

Formula: X(ertriage)


,case:concept:name,accepted
0,A,False
1,B,True
2,C,True
3,D,True
4,E,True
...,...,...
1045,HNA,True
1046,INA,True
1047,JNA,True
1048,KNA,True


### `eventually_a_then_b`

This is a binary template taking one activity as input. The corresponding LTL formula is `F(a && F(b))`.

In [7]:
template = LTLTemplate('eventually_a_then_b')
model = template.fill_template(['Leucocytes', 'CRP'])
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
data

Formula: F(leucocytes && F(crp))


,case:concept:name,accepted
0,A,True
1,B,True
2,C,True
3,D,True
4,E,False
...,...,...
1045,HNA,True
1046,INA,False
1047,JNA,False
1048,KNA,True


### `eventually_a_or_b`

This is a binary template taking one activity as input. The corresponding LTL formula is `F(a) || F(b)`.

In [6]:
template = LTLTemplate('eventually_a_or_b')
model = template.fill_template(['Leucocytes', 'CRP'])
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
data

Formula: F(leucocytes) || F(crp)


,case:concept:name,accepted
0,A,True
1,B,True
2,C,True
3,D,True
4,E,True
...,...,...
1045,HNA,True
1046,INA,False
1047,JNA,False
1048,KNA,True


### `eventually_a_next_b`

This is a binary template taking one activity as input. The corresponding LTL formula is `F(a && X(b))`.

In [12]:
template = LTLTemplate('eventually_a_next_b')
model = template.fill_template(['Leucocytes', 'CRP'])
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
data

Formula: F(leucocytes && X(crp))


,case:concept:name,accepted
0,A,True
1,B,False
2,C,True
3,D,True
4,E,False
...,...,...
1045,HNA,True
1046,INA,False
1047,JNA,False
1048,KNA,True


### `eventually_a_then_b_then_c`

This is a ternary template taking one activity as input. The corresponding LTL formula is `F(a && F(b && F(c)))`.

In [13]:
template = LTLTemplate('eventually_a_then_b_then_c')
model = template.fill_template(['ER Registration', 'Leucocytes', 'CRP'])
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
data

Formula: F(erregistration && F(leucocytes && F(crp)))


,case:concept:name,accepted
0,A,True
1,B,True
2,C,True
3,D,True
4,E,False
...,...,...
1045,HNA,True
1046,INA,False
1047,JNA,False
1048,KNA,True


### `eventually_A_next_B_next_C`

This is a ternary template taking one activity as input. The corresponding LTL formula is `F(a && X(b && X(c)))`.

In [14]:
template = LTLTemplate('eventually_a_next_b_next_c')
model = template.fill_template(['ER Registration', 'CRP', 'Leucocytes'])
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
data

Formula: F(erregistration && X(crp && X(leucocytes)))


,case:concept:name,accepted
0,A,False
1,B,False
2,C,False
3,D,False
4,E,False
...,...,...
1045,HNA,False
1046,INA,False
1047,JNA,False
1048,KNA,False


## Conformance Checking with Target-Branched DECLARE templates

A series of templates for [Target-Branched DECLARE](https://www.sciencedirect.com/science/article/pii/S0306437915001271). All functions take two lists of strings as parameters. The first parameter is the source while the second list is the target.

Hereafter, we consider the symbols `s1, s2, ... sn` the source activities and `t1, t2, ... tn` the target activities.

### `responded_existence`

The corresponding LTL formula of this binary template is `F(s1 || s2 || ... -> F(t1 || t2 || ... ))`.

In [6]:
template = LTLTemplate('responded_existence')
source_list = ["ER Triage", "CRP", "Leucocytes"]
target_list = ["Leucocytes", "Admission NC", "Release A"]
model = template.fill_template(source_list, target_list)
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
data

Formula: F(ertriage || crp || leucocytes) -> F(leucocytes || admissionnc || releasea)


,case:concept:name,accepted
0,A,True
1,B,True
2,C,True
3,D,True
4,E,True
...,...,...
1045,HNA,True
1046,INA,False
1047,JNA,False
1048,KNA,True


### `response`

The corresponding LTL formula of this binary template is `G(s1 || s2 || ... -> F(t1 || t2 || ... ))`.

In [20]:
template = LTLTemplate('response')
source_list = ["ER Triage", "CRP", "Leucocytes"]
target_list = ["Leucocytes", "Admission NC", "Release A"]
model = template.fill_template(source_list, target_list)
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
data

Formula: G(ertriage -> F(leucocytes))


,case:concept:name,accepted
0,A,True
1,B,True
2,C,True
3,D,True
4,E,True
...,...,...
1045,HNA,True
1046,INA,False
1047,JNA,False
1048,KNA,True


### `alternate_response`

The corresponding LTL formula of this binary template is `G(s1 || s2 || ... -> X((!(s1) || !(s2) || ... ))U(t1 || t2 || ...)))`

In [7]:
template = LTLTemplate('alternate_response')
source_list = ["ER Triage", "CRP", "Leucocytes"]
target_list = ["Leucocytes", "Admission NC", "Release A"]
model = template.fill_template(source_list, target_list)
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
data

Formula: G(ertriage || crp || leucocytes -> X((!(ertriage) ||  !(crp) ||  !(leucocytes))U(leucocytes || admissionnc || releasea)))


,case:concept:name,accepted
0,A,True
1,B,True
2,C,True
3,D,True
4,E,False
...,...,...
1045,HNA,True
1046,INA,False
1047,JNA,False
1048,KNA,True


### `chain_response`

The corresponding LTL formula of this binary template is `G(s1 || s2 || ... -> X(t1 || t2 || ... ))`.

In [17]:
template = LTLTemplate('chain_response')
source_list = ["ER Registration", "CRP"]
target_list = ["ER Triage", "Leucocytes", "Release A"]
model = template.fill_template(source_list, target_list)
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
data

Formula: G(erregistration  || crp -> X(ertriage || leucocytes || releasea))


,case:concept:name,accepted
0,A,False
1,B,False
2,C,False
3,D,False
4,E,True
...,...,...
1045,HNA,False
1046,INA,True
1047,JNA,True
1048,KNA,False


### `precedence`

The corresponding LTL formula of this binary template is `((!(leucocytes)||  !(admissionnc)||  !(releasea))U(ertriage|| crp|| leucocytes)) || G((!(admissionnc)|| !(admissionnc)|| !(releasea)))`

In [14]:
template = LTLTemplate('precedence')
target_list = ["ER Triage", "CRP"]
#target_list = ["Leucocytes", "Admission NC", "Release A"]
source_list = ["ER Registration", "CRP"]
model = template.fill_template(source_list, target_list)
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
data

Formula: ((!(ertriage)||  !(crp))U(erregistration|| crp)) || G((!(crp)|| !(crp)))


,case:concept:name,accepted
0,A,True
1,B,True
2,C,True
3,D,True
4,E,True
...,...,...
1045,HNA,True
1046,INA,True
1047,JNA,True
1048,KNA,True


### `alternate_precedence([A_1 or A_2 or ...], [B_1 or B_2 or ...])`

In [20]:
template = LTLModelTemplate('alternate_precedence')
source_list = ["ER Triage", "CRP", "Leucocytes"]
target_list = ["Leucocytes", "Admission NC", "Release A"]
model = template.get_templ_model(source_list, target_list)
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

    case id  accepting
0         0       True
0         1       True
0         2       True
0         3       True
0         4       True
..      ...        ...
0      1045       True
0      1046       True
0      1047       True
0      1048       True
0      1049       True

[1050 rows x 2 columns]


### `chain_precedence([A_1 or A_2 or ...], [B_1 or B_2 or ...])`

In [21]:
template = LTLModelTemplate('chain_precedence')
source_list = ["ER Triage", "CRP", "Leucocytes"]
target_list = ["Leucocytes", "Admission NC", "Release A"]
model = template.get_templ_model(source_list, target_list)
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

    case id  accepting
0         0      False
0         1      False
0         2      False
0         3      False
0         4      False
..      ...        ...
0      1045      False
0      1046      False
0      1047      False
0      1048      False
0      1049      False

[1050 rows x 2 columns]


### `not_responded_existence([A_1 or A_2 or ...], [B_1 or B_2 or ...])`

In [22]:
template = LTLModelTemplate('not_responded_existence')
source_list = ["ER Triage", "CRP", "Leucocytes"]
target_list = ["Leucocytes", "Admission NC", "Release A"]
model = template.get_templ_model(source_list, target_list)
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

    case id  accepting
0         0      False
0         1      False
0         2      False
0         3      False
0         4      False
..      ...        ...
0      1045      False
0      1046       True
0      1047       True
0      1048      False
0      1049       True

[1050 rows x 2 columns]


### `not_response([A_1 or A_2 or ...], [B_1 or B_2 or ...])`

In [23]:
template = LTLModelTemplate('not_response')
source_list = ["ER Triage", "CRP", "Leucocytes"]
target_list = ["Leucocytes", "Admission NC", "Release A"]
model = template.get_templ_model(source_list, target_list)
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

    case id  accepting
0         0      False
0         1      False
0         2      False
0         3      False
0         4       True
..      ...        ...
0      1045      False
0      1046       True
0      1047       True
0      1048      False
0      1049       True

[1050 rows x 2 columns]


### `not_precedence([A_1 or A_2 or ...], [B_1 or B_2 or ...])`

In [24]:
template = LTLModelTemplate('not_precedence')
source_list = ["ER Triage", "CRP", "Leucocytes"]
target_list = ["Leucocytes", "Admission NC", "Release A"]
model = template.get_templ_model(source_list, target_list)
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

    case id  accepting
0         0      False
0         1      False
0         2      False
0         3      False
0         4      False
..      ...        ...
0      1045      False
0      1046       True
0      1047       True
0      1048      False
0      1049       True

[1050 rows x 2 columns]


### `not_chain_response([A_1 or A_2 or ...], [B_1 or B_2 or ...])`

In [25]:
template = LTLModelTemplate('not_chain_response')
source_list = ["ER Triage", "CRP", "Leucocytes"]
target_list = ["Leucocytes", "Admission NC", "Release A"]
model = template.get_templ_model(source_list, target_list)
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

    case id  accepting
0         0       True
0         1       True
0         2       True
0         3       True
0         4       True
..      ...        ...
0      1045       True
0      1046       True
0      1047       True
0      1048       True
0      1049       True

[1050 rows x 2 columns]


### `not_chain_precedence`

The corresponding LTL formula of this binary template is `G(X(t1 || t2 || ...) ->  !(s1) || !(s2)|| ... `

In [18]:
template = LTLTemplate('not_chain_precedence')
source_list = ["ER Triage", "CRP", "Leucocytes"]
target_list = ["Leucocytes", "Admission NC", "Release A"]
model = template.fill_template(source_list, target_list)
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
data

Formula: G(X(leucocytes) ->  !(ertriage))


,case:concept:name,accepted
0,A,True
1,B,True
2,C,True
3,D,True
4,E,True
...,...,...
1045,HNA,True
1046,INA,True
1047,JNA,True
1048,KNA,True
